# Trying to load Torch BERT weights into Jax

Reproduces shape error

In [1]:
import sys

In [2]:
sys.path.append('../')
sys.path.append('../bert')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import jax.numpy as jnp
import numpy as np

/home/mike/miniconda3/envs/JaxTestAgain/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from bert.explanation_generator import Generator

In [6]:
from bert.loading_utils import flax_bert_params_from_pt

In [7]:
from bert.modeling_flax_bert import FlaxBertForSequenceClassification

In [8]:
state_dict = torch.load('./classifier.pt', map_location=torch.device('cpu'))

In [9]:
bert_flax = FlaxBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing FlaxBertForSequenceClassification: {('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias')}
- This IS expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of FlaxBertForSequenceClassification were not initialized from the model checkpoint at bert-base-unca

In [10]:
bert_flax.params = flax_bert_params_from_pt(state_dict, 768, 12)

/home/mike/repos/JAX-Transformer-Explainability/nb/../bert/loading_utils.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  val = val.T


In [11]:
generator = Generator(bert_flax)

In [12]:
attention_mask = jnp.load('demo_attention_mask.npz')['arr_0']

In [13]:
input_ids = jnp.load('demo_input_ids.npz')['arr_0']

In [14]:
generator.generate_LRP(input_ids, attention_mask, index=0)

ScopeParamShapeError: Inconsistent shapes between value and initializer for parameter "embedding" in "/bert/embeddings/position_embeddings": (1, 512, 768), (512, 768). (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ScopeParamShapeError)